In [1]:
import cv2 as cv
import numpy as np

In [ ]:
path = "/home/pks/Downloads/Assignment/IVP/mini project/"

In [29]:
import cv2 as cv
import numpy as np
import imutils

path = "/home/pks/Downloads/Assignment/IVP/mini project/"

def f(c, th):
    contours_poly = cv.approxPolyDP(c, 3, True)
    boundRect = cv.boundingRect(contours_poly)
    
    return boundRect

def number_extraction(image):
    
#     image = cv.GaussianBlur(image, (3,3), 0)
    cv.imshow("img", image)
    cv.waitKey(0)

    _, thresh = cv.threshold(image, 0, 255, cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
#     thresh = cv.adaptiveThreshold(image, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY_INV, 11, 2)
#     thresh = cv.Canny(image, 80, 120)
    cv.imshow("thresh", thresh)
    cv.waitKey(0)
    
#     kernel = np.ones((3,3), np.uint8)
#     thresh = cv.dilate(thresh, kernel, iterations=1)
#     cv.imshow("thresh", thresh)
#     cv.waitKey(0)
#     thresh = cv.erode(thresh, kernel, iterations=1)
#     cv.imshow("thresh", thresh)
#     cv.waitKey(0)

    cnts = cv.findContours(thresh.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    drawing = image.copy()
    rect = list()
    for c_el in cnts:
        boundRect = f(c_el, thresh)
        area, wh_ratio = boundRect[2]*boundRect[3], boundRect[2]/boundRect[3] 
        if area > 80 and area < 300 and wh_ratio < 3.5 and wh_ratio > 0.28:
#             print(cv.contourArea(c_el))
    #         print(boundRect)
            rect.append([(int(boundRect[0]), int(boundRect[1])), (int(boundRect[0]+boundRect[2]), int(boundRect[1]+boundRect[3]))])

            cv.rectangle(drawing, (int(boundRect[0]), int(boundRect[1])), (int(boundRect[0]+boundRect[2]), int(boundRect[1]+boundRect[3])), 0, 1)

    
    print(rect)
    
    left = min(rect, key=lambda x: x[0][0])[0][0]
    right = max(rect, key=lambda x: x[1][0])[1][0]
    top = min(rect, key=lambda x: x[0][1])[0][1]
    bottom = max(rect, key=lambda x: x[1][1])[1][1]
#     lr_coord = sorted(rect, key=lambda x: (x[0][0], x[1][0]))
#     tb_coord = sorted(rect, key=lambda x: (x[0][1], x[1][1]))
    
#     left, right, top, bottom = lr_coord[0][0][0], lr_coord[-1][1][0], tb_coord[0][0][1], tb_coord[-1][1][1]
    print("i")
    print(top, bottom, left, right)
    
    cv.imshow("wrap", drawing)
    cv.waitKey(0)
    
    drawing = image.copy()
    ret_val = []
#     print(cnts)
    for c_el in cnts:
        boundRect = f(c_el, thresh)
        start, end = (int(boundRect[0]), int(boundRect[1])), (int(boundRect[0]+boundRect[2]), int(boundRect[1]+boundRect[3]))
        print("start", start, "end", end)
        if top <= start[1] and bottom >= end[1] and left <= start[0] and right >= end[0]:
            print(start, end)
            cv.rectangle(drawing, start, end, 0, 1)
            ret_val.append([(int(boundRect[0]), int(boundRect[1])), (int(boundRect[0]+boundRect[2]), int(boundRect[1]+boundRect[3]))])
        
            cv.imshow("wrap", drawing)
            cv.waitKey(0)
    
    return ret_val, image

image = cv.imread(path+"img20.jpg", 0)
ret, thresh = number_extraction(image.copy())

ret = sorted(ret, key=lambda x: (x[0], x[1]))
counter = 0
for r in ret:
    temp = thresh[r[0][1]:r[1][1], r[0][0]:r[1][0]]
    cv.imshow("temp", temp)
    cv.waitKey(0)
    cv.imwrite(path + "i" + str(counter) + ".jpg", temp)
    counter = counter + 1


cv.destroyAllWindows()

[[(8, 17), (20, 33)], [(22, 15), (33, 34)]]
i
15 34 8 33
start (8, 17) end (20, 33)
(8, 17) (20, 33)
start (22, 15) end (33, 34)
(22, 15) (33, 34)
start (43, 1) end (44, 2)
start (31, 1) end (38, 2)
start (27, 1) end (28, 2)
start (46, 0) end (47, 41)
start (0, 0) end (26, 2)


In [28]:
cv.GaussianBlur

<function GaussianBlur>